# 1. Candidate Generation
### 데이터 읽기 및 전처리

In [12]:
from urllib.request import urlretrieve
import zipfile
import pandas as pd
import numpy as np
import random

# data 읽기
# ------------- user columns ----------------------------
users_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']


# read data
users = pd.read_csv(
    'data/u.user', sep='|', names=users_cols, encoding='latin-1')


# ------------- watch columns ----------------------------
watch_cols = ['user_id', 'movie_id', 'watch_hist_time']


watches = pd.read_csv(
    'data/u.watch', sep='\t', names=watch_cols, encoding='latin-1')



# ------------- search columns ----------------------------
search_cols = ['user', 'search_hist']
searches = pd.read_csv(
    'data/u.search', sep='\t', names=search_cols, encoding='latin-1')

# The movies file contains a binary feature for each genre.
genre_cols = [
    "genre_unknown", "Action", "Adventure", "Animation", "Children", "Comedy",
    "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror",
    "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western",
]

# ------------- movies columns ----------------------------
movies_cols = [
    'movie_id', 'title', 'release_date', "video_release_date", "imdb_url"
] + genre_cols
movies = pd.read_csv(
    'data/u.item', sep='|', names=movies_cols, encoding='latin-1')

# Since the ids start at 1, we shift them to start at 0.
# DON'T SHIFT TO START AT 0 'CAUSE WE WILL FAIL TO CONVERT TYPE LATER
# users["user_id"] = users["user_id"].apply(lambda x: str(x-1))
# movies["movie_id"] = movies["movie_id"].apply(lambda x: str(x-1))
movies["year"] = movies['release_date'].apply(lambda x: str(x).split('-')[-1])
# watches["movie_id"] = watches["movie_id"].apply(lambda x: str(x-1))
# watches["user_id"] = watches["user_id"].apply(lambda x: str(x-1))
# searches["user_id"] = searches["user"].apply(lambda x: str(x-1))

# example_age 추가
movies['example_age'] = (pd.to_datetime("now") - pd.to_datetime(movies['release_date']))\
            /np.timedelta64(1,'D') 

# normalize
# norm_val = (val - min)/(max-min) 
# because max-min is the largest number, any value between max & min divided by max-min is always in [0,1]
def normalize_col(df,col_name):
    df[col_name] = (df[col_name] - df[col_name].min()) / (df[col_name].max() - df[col_name].min())
    return df

movies = normalize_col(movies,'example_age')
watches = normalize_col(watches,'watch_hist_time')


# data 합치기
# WILL FAIL IF SHIFTED AT 0 HERE
data = watches.merge(movies, on='movie_id').merge(users, on='user_id')
data['user_id']=data['user_id'].astype(int)
data['movie_id']=data['movie_id'].astype(int)
data = data.set_index(['user_id']).sort_index()
data = data.reset_index()
data['movie_name']=data['title'].str[:-6] # 년도 부분 자르기


# occupation 인코딩
occupations = data["occupation"].unique().tolist()
occupations_encoded = {x: i for i, x in enumerate(occupations)}
occupationsencoded2occupations = {i: x for i, x in enumerate(occupations)}

# search history 인코딩
search_hists = searches["search_hist"].unique().tolist()
search_encoded = {x: i for i, x in enumerate(search_hists)}
searchencoded2search = {i: x for i, x in enumerate(search_hists)}

# User index encoding
user_ids = data["user_id"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}

# movie index encoding
movie_ids = data["movie_id"].unique().tolist()
movie2movie_encoded = {x: i for i, x in enumerate(movie_ids)}
movie_encoded2movie = {i: x for i, x in enumerate(movie_ids)}

# movie title encoding
title_ids = data["title"].unique().tolist()
title2title_encoded = {x: i for i, x in enumerate(title_ids)}
title_encoded2title = {i: x for i, x in enumerate(title_ids)}

# change to encoding
data["user"] = data["user_id"].map(user2user_encoded)
data["movie"] = data["movie_id"].map(movie2movie_encoded)
data["title_d"] = data["title"].map(title2title_encoded)
searches["search_hist"] = searches["search_hist"].map(search_encoded)
data["occupation"] = data["occupation"].map(occupations_encoded)
# searches["search_hist"] = searches["search_hist"]
searches = normalize_col(searches,'search_hist')

watch_hist = data.groupby(['user'])['movie_id'].apply(list).reset_index()
search_hist = searches.groupby(['user'])['search_hist'].apply(list).reset_index()
watch_hist_time = data.groupby(['user'])['watch_hist_time'].apply(list).reset_index()
example_age = data.groupby(['user'])['example_age'].apply(list).reset_index()

user_video_list = data.pivot(index='user_id', columns='movie_id', values='movie').reset_index()
user_video_list.fillna(data["movie_id"].max()+1, inplace=True)

sample_data=data[['user','occupation','sex']]
sample_data=sample_data.reset_index()
sample_data = sample_data.drop('index',axis=1)
sample_data = sample_data.drop_duplicates()

user_movie_list = pd.merge(sample_data,watch_hist, how= 'left')
user_movie_list = pd.merge(user_movie_list,watch_hist_time, how='left')
user_movie_list = pd.merge(user_movie_list,search_hist, how='left')
user_movie_list = pd.merge(user_movie_list,example_age, how='left')
user_movie_list['search_hist'] = user_movie_list['search_hist'].apply(lambda x: x if type(x) is list else []) # NaN 처리
user_movie_list['predict_labels'] = user_movie_list['movie_id'].apply(lambda x: int(random.uniform(0,data["movie"].max()))) #label을 마지막 값으로..



train_data = user_movie_list[(user_movie_list.user >= 1)&
                                  (user_movie_list.user <= 5)]
test_data = user_movie_list[(user_movie_list.user >= 6)&
                                  (user_movie_list.user <= 10)]





In [13]:
# movies # 영화 정보 데이터
user_movie_list

,user,occupation,sex,movie_id,watch_hist_time,search_hist,example_age,predict_labels
0,0,0,M,"[10, 3, 9, 5]","[0.024163568773234202, 0.0, 0.2131350681536555...",[0.0],"[0.9577264264593144, 0.0, 1.0, 0.8799693352316...",8
1,1,1,F,"[2, 7]","[0.007390688617454417, 0.1644538856434767]","[0.0, 1.0]","[0.3599824772752163, 0.07994743182564888]",2
2,2,2,M,"[3, 8]","[0.17919100725792175, 0.2091520623119136]",[0.5],"[0.0, 1.0]",0
3,3,0,M,[8],[1.0],[1.0],[1.0],4
4,4,1,F,[4],[0.017923526287838557],[],[1.0],4
5,5,3,M,[6],[0.1614887590724022],[],[1.0],6
6,6,4,M,"[2, 8]","[0.4109134360063728, 0.17919100725792175]",[],"[0.3599824772752163, 1.0]",6
7,7,4,M,"[7, 10, 1]","[0.20477075588599752, 0.1644538856434767, 0.03...",[0.0],"[0.07994743182564888, 0.9577264264593144, 1.0]",5
8,8,5,M,[1],[0.4109134360063728],[],[1.0],5
9,9,6,M,[1],[0.20069923880332802],[],[1.0],7


In [14]:
train_data # train data

,user,occupation,sex,movie_id,watch_hist_time,search_hist,example_age,predict_labels
1,1,1,F,"[2, 7]","[0.007390688617454417, 0.1644538856434767]","[0.0, 1.0]","[0.3599824772752163, 0.07994743182564888]",2
2,2,2,M,"[3, 8]","[0.17919100725792175, 0.2091520623119136]",[0.5],"[0.0, 1.0]",0
3,3,0,M,[8],[1.0],[1.0],[1.0],4
4,4,1,F,[4],[0.017923526287838557],[],[1.0],4
5,5,3,M,[6],[0.1614887590724022],[],[1.0],6


### Hyperparameter Definition

In [15]:
# CONTINUE FROM HERE - 30-09-2021
EMBEDDING_DIMS = 16
DENSE_UNITS = 64
DROPOUT_PCT = 0.0
ALPHA = 0.0
NUM_CLASSES=data["movie"].max() + 2
LEARNING_RATE = 0.003
tf.random.set_seed(
    1324
)

### custom layers 

In [16]:
import tensorflow as tf
class MaskedEmbeddingsAggregatorLayer(tf.keras.layers.Layer):
    def __init__(self, agg_mode='sum', **kwargs):
        super(MaskedEmbeddingsAggregatorLayer, self).__init__(**kwargs)

        if agg_mode not in ['sum', 'mean']:
            raise NotImplementedError('mode {} not implemented!'.format(agg_mode))
        self.agg_mode = agg_mode
    
    @tf.function
    def call(self, inputs, mask=None):
#         https://www.tensorflow.org/api_docs/python/tf/ragged/boolean_mask
        masked_embeddings = tf.ragged.boolean_mask(inputs, mask)
        if self.agg_mode == 'sum':
            aggregated =  tf.reduce_sum(masked_embeddings, axis=1)
        elif self.agg_mode == 'mean':
            aggregated = tf.reduce_mean(masked_embeddings, axis=1)
        return aggregated
    
    def get_config(self):
        # this is used when loading a saved model that uses a custom layer
        return {'agg_mode': self.agg_mode}
    
class L2NormLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(L2NormLayer, self).__init__(**kwargs)
    
    @tf.function
    def call(self, inputs, mask=None):
        if mask is not None:
            inputs = tf.ragged.boolean_mask(inputs, mask).to_tensor()
        return tf.math.l2_normalize(inputs, axis=-1)

    def compute_mask(self, inputs, mask):
        return mask


### model

In [17]:
#---inputs
import tensorflow as tf
import datetime
import os
input_watch_hist = tf.keras.Input(shape=(None, ), name='watch_hist')
input_watch_hist_time = tf.keras.layers.Input(shape=(None,), name='watch_hist_time')
input_search_hist = tf.keras.layers.Input(shape=(None,), name='search_hist')
input_example_age = tf.keras.Input(shape=(None, ), name='example_age')
input_occupation = tf.keras.Input(shape=(None, ), name='occupation')


#--- layers
features_embedding_layer = tf.keras.layers.Embedding(input_dim=NUM_CLASSES, output_dim=EMBEDDING_DIMS, 
                                            mask_zero=True, trainable=True, name='features_embeddings')
labels_embedding_layer = tf.keras.layers.Embedding(input_dim=NUM_CLASSES, output_dim=EMBEDDING_DIMS, 
                                            mask_zero=True, trainable=True, name='labels_embeddings')

avg_embeddings = MaskedEmbeddingsAggregatorLayer(agg_mode='mean', name='aggregate_embeddings')

dense_1 = tf.keras.layers.Dense(units=DENSE_UNITS, name='dense_1')
dense_2 = tf.keras.layers.Dense(units=DENSE_UNITS, name='dense_2')
dense_3 = tf.keras.layers.Dense(units=DENSE_UNITS, name='dense_3')
l2_norm_1 = L2NormLayer(name='l2_norm_1')

dense_output = tf.keras.layers.Dense(NUM_CLASSES, activation=tf.nn.softmax, name='dense_output')

#--- features
features_embeddings = features_embedding_layer(input_watch_hist)
l2_norm_features = l2_norm_1(features_embeddings)
avg_features = avg_embeddings(l2_norm_features)

labels_watch_embeddings = labels_embedding_layer(input_watch_hist_time)
l2_norm_watched = l2_norm_1(labels_watch_embeddings)
avg_watched = avg_embeddings(l2_norm_watched)

labels_search_embeddings = labels_embedding_layer(input_search_hist)
l2_norm_searched = l2_norm_1(labels_search_embeddings)
avg_searched = avg_embeddings(l2_norm_searched)

labels_example_age_embeddings = labels_embedding_layer(input_example_age)
l2_norm_example_age = l2_norm_1(labels_example_age_embeddings)
avg_example_age = avg_embeddings(l2_norm_example_age)

labels_occupation_embeddings = labels_embedding_layer(input_occupation)
l2_norm_occupation = l2_norm_1(labels_occupation_embeddings)
avg__occupation = avg_embeddings(l2_norm_occupation)


print(avg_features)
print(avg_watched)
print(avg_searched)
print(avg_example_age)
print(input_occupation)

# 임베딩 벡터들 연결
concat_inputs = tf.keras.layers.Concatenate(axis=1)([avg_features,
                                                     avg_watched,
                                                     avg_searched,
                                                     avg_example_age,
#                                                      avg__occupation
                                                     ])
# Dense Layers
dense_1_features = dense_1(concat_inputs)
dense_1_relu = tf.keras.layers.ReLU(name='dense_1_relu')(dense_1_features)
dense_1_batch_norm = tf.keras.layers.BatchNormalization(name='dense_1_batch_norm')(dense_1_relu)

dense_2_features = dense_2(dense_1_relu)
dense_2_relu = tf.keras.layers.ReLU(name='dense_2_relu')(dense_2_features)
# dense_2_batch_norm = tf.keras.layers.BatchNormalization(name='dense_2_batch_norm')(dense_2_relu)

dense_3_features = dense_3(dense_2_relu)
dense_3_relu = tf.keras.layers.ReLU(name='dense_3_relu')(dense_3_features)
dense_3_batch_norm = tf.keras.layers.BatchNormalization(name='dense_3_batch_norm')(dense_3_relu)
outputs = dense_output(dense_3_batch_norm)

#Optimizer
optimiser = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

#--- prep model
model = tf.keras.models.Model(
    inputs=[input_watch_hist, 
            input_watch_hist_time, 
            input_search_hist,
            input_example_age,
#             input_occupation,
            ],
    outputs=[outputs]
)
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
model.compile(optimizer=optimiser, loss='sparse_categorical_crossentropy', metrics=['acc'])

model.summary()

KerasTensor(type_spec=TensorSpec(shape=(None, 16), dtype=tf.float32, name=None), name='aggregate_embeddings/PartitionedCall:0', description="created by layer 'aggregate_embeddings'")
KerasTensor(type_spec=TensorSpec(shape=(None, 16), dtype=tf.float32, name=None), name='aggregate_embeddings/PartitionedCall:0', description="created by layer 'aggregate_embeddings'")
KerasTensor(type_spec=TensorSpec(shape=(None, 16), dtype=tf.float32, name=None), name='aggregate_embeddings/PartitionedCall:0', description="created by layer 'aggregate_embeddings'")
KerasTensor(type_spec=TensorSpec(shape=(None, 16), dtype=tf.float32, name=None), name='aggregate_embeddings/PartitionedCall:0', description="created by layer 'aggregate_embeddings'")
KerasTensor(type_spec=TensorSpec(shape=(None, None), dtype=tf.float32, name='occupation'), name='occupation', description="created by layer 'occupation'")
Model: "model_1"
________________________________________________________________________________________________

### Starts learning

In [18]:
train_data

,user,occupation,sex,movie_id,watch_hist_time,search_hist,example_age,predict_labels
1,1,1,F,"[2, 7]","[0.007390688617454417, 0.1644538856434767]","[0.0, 1.0]","[0.3599824772752163, 0.07994743182564888]",2
2,2,2,M,"[3, 8]","[0.17919100725792175, 0.2091520623119136]",[0.5],"[0.0, 1.0]",0
3,3,0,M,[8],[1.0],[1.0],[1.0],4
4,4,1,F,[4],[0.017923526287838557],[],[1.0],4
5,5,3,M,[6],[0.1614887590724022],[],[1.0],6


In [19]:
history = model.fit([tf.keras.preprocessing.sequence.pad_sequences(train_data['movie_id']),
           tf.keras.preprocessing.sequence.pad_sequences(train_data['watch_hist_time'], dtype=float),
           tf.keras.preprocessing.sequence.pad_sequences(train_data['search_hist'], dtype=float) + 1e-10,
           tf.keras.preprocessing.sequence.pad_sequences(train_data['example_age'], dtype=float),
#            tf.keras.preprocessing.sequence.pad_sequences(train_data['occupation'], dtype=float),
           ],train_data['predict_labels'].values,
           steps_per_epoch=1, epochs=50)




Epoch 1/50
1/1 [==============================] - 1s 1s/step - loss: 3.0103 - acc: 0.0000e+00
Epoch 2/50
1/1 [==============================] - 0s 6ms/step - loss: 2.1642 - acc: 0.2000
Epoch 3/50
1/1 [==============================] - 0s 5ms/step - loss: 1.6725 - acc: 0.4000
Epoch 4/50
1/1 [==============================] - 0s 5ms/step - loss: 1.3902 - acc: 0.8000
Epoch 5/50
1/1 [==============================] - 0s 4ms/step - loss: 1.1259 - acc: 0.8000
Epoch 6/50
1/1 [==============================] - 0s 3ms/step - loss: 0.9581 - acc: 0.8000
Epoch 7/50
1/1 [==============================] - 0s 3ms/step - loss: 0.8353 - acc: 0.8000
Epoch 8/50
1/1 [==============================] - 0s 4ms/step - loss: 0.7353 - acc: 0.8000
Epoch 9/50
1/1 [==============================] - 0s 3ms/step - loss: 0.6462 - acc: 0.8000
Epoch 10/50
1/1 [==============================] - 0s 3ms/step - loss: 0.5718 - acc: 0.8000
Epoch 11/50
1/1 [==============================] - 0s 4ms/step - loss: 0.5163 - acc: 0

In [20]:
model.save("candidate_generation.h5")

c:\python39\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [21]:
pred = model.predict([tf.keras.preprocessing.sequence.pad_sequences(test_data['movie_id']),
           tf.keras.preprocessing.sequence.pad_sequences(test_data['watch_hist_time'], dtype=float),
           tf.keras.preprocessing.sequence.pad_sequences(test_data['search_hist'], dtype=float) + 1e-10,
           tf.keras.preprocessing.sequence.pad_sequences(test_data['example_age'], dtype=float)
           ])

In [22]:
pred

array([[0.08890043, 0.05966489, 0.05075201, 0.06509386, 0.19279505,
        0.05343901, 0.2623308 , 0.0601544 , 0.05396948, 0.05052129,
        0.06237875],
       [0.06356684, 0.07987908, 0.14043802, 0.08706639, 0.09777406,
        0.06319581, 0.15613799, 0.08463792, 0.07409474, 0.06471342,
        0.08849567],
       [0.02777404, 0.03783209, 0.03512511, 0.03637972, 0.35174787,
        0.03023753, 0.35216188, 0.03147651, 0.03207491, 0.03208072,
        0.03310964],
       [0.02777404, 0.03783209, 0.03512511, 0.03637972, 0.35174787,
        0.03023753, 0.35216188, 0.03147651, 0.03207491, 0.03208072,
        0.03310964]], dtype=float32)

In [23]:
# candidate generation: 
###### We extract the top-7 recommendation data for each user.
N = 6
k = np.sort((-pred).argsort()[:,:N])
print(k)
k = k.flatten()
k[k>data["movie"].max()]=0
k = np.unique(k)


[[ 0  3  4  6  7 10]
 [ 2  3  4  6  7 10]
 [ 1  2  3  4  6 10]
 [ 1  2  3  4  6 10]]


# 2. Ranking

### Load model and preprocess data

In [24]:
# load candidate_generation 
model = tf.keras.models.load_model(
    'candidate_generation.h5',
    custom_objects={
        'L2NormLayer':L2NormLayer,
        'MaskedEmbeddingsAggregatorLayer':MaskedEmbeddingsAggregatorLayer
    }
)

In [25]:
def get_genres(movies, genres):
	def get_all_genres(gs):
		active = [str(genres_encoded[genre]) for genre, g in zip(genres, gs) if g==1]
		if len(active) == 0:
			return '0'
		return ','.join((active))
	
	movies['all_genres'] = [
	get_all_genres(gs) for gs in zip(*[movies[genre] for genre in genres])]


In [26]:
movie_data = movies.set_index(['movie_id']).sort_index()
movie_data = movie_data.loc[k+1]
movie_data["title_d"] = movie_data["title"].map(title2title_encoded)

In [27]:

ratings_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(
    'data/u.data', sep='\t', names=ratings_cols, encoding='latin-1')

genres_encoded = {x: i for i, x in enumerate(genre_cols)}
get_genres(movie_data, genre_cols)

new_data = movie_data.merge(ratings, on='movie_id') # rating 추가

genre_occurences = new_data[genre_cols].sum().to_dict()
genre_occurences


{'genre_unknown': 0,
 'Action': 3,
 'Adventure': 2,
 'Animation': 3,
 'Children': 6,
 'Comedy': 7,
 'Crime': 1,
 'Documentary': 0,
 'Drama': 7,
 'Fantasy': 0,
 'Film-Noir': 0,
 'Horror': 0,
 'Musical': 0,
 'Mystery': 0,
 'Romance': 0,
 'Sci-Fi': 2,
 'Thriller': 5,
 'War': 0,
 'Western': 0}

In [28]:

new_data = new_data[['movie_id', 'user_id', 'rating', 'unix_timestamp', 'all_genres', 'title_d']]
new_data['movie_type'] = np.where(new_data['rating'] >= 3, 'like', 'dislike') # 3보다 크면 like
new_data['movie_type']

0     dislike
1     dislike
2        like
3        like
4        like
5     dislike
6        like
7     dislike
8        like
9        like
10       like
11       like
12       like
13    dislike
Name: movie_type, dtype: object

In [29]:

movie_list = new_data.groupby(['user_id','movie_type'])['movie_id'].apply(list).reset_index()

movie_list

,user_id,movie_type,movie_id
0,1,dislike,[3]
1,1,like,[5]
2,2,like,"[2, 7]"
3,3,like,"[3, 8]"
4,4,dislike,[8]
5,5,dislike,[4]
6,7,like,"[2, 8]"
7,8,like,"[1, 7]"
8,9,dislike,[1]
9,10,dislike,[1]


In [30]:

genre_list = new_data.groupby(['user_id'])['all_genres'].unique().apply(list).reset_index()
genre_list['all_genres']=genre_list['all_genres'].apply(lambda x: list(set(','.join(x))) ) # 중복제거
genre_list['all_genres']=genre_list['all_genres'].apply(lambda x:[ x for x in x if x.isdigit() ])

new_data = normalize_col(new_data, 'unix_timestamp')
timestamp_list = new_data.groupby(['user_id'])['unix_timestamp'].unique().apply(list).reset_index()

title_list = new_data.groupby(['user_id'])['title_d'].apply(list).reset_index()
print(movie_list)
dataset = movie_list.pivot(index='user_id', columns='movie_type', values='movie_id').reset_index()
dataset.fillna(new_data["movie_id"].max()+1, inplace=True)

dataset['like'] =dataset['like'].apply(lambda x: x if type(x) is list else [])
dataset['dislike'] =dataset['dislike'].apply(lambda x: x if type(x) is list else [])

dataset = pd.merge(dataset, title_list, how='left')
dataset = pd.merge(dataset, genre_list, how='left')
dataset = pd.merge(dataset, timestamp_list, how='left')

dataset['predict_labels'] = dataset['like'].apply(lambda x: int(random.uniform(1,new_data["movie_id"].max()))) #label을 마지막 값으로..

dataset['like']=dataset['like'].apply(lambda x: [new_data["movie_id"].max()+1] if x == [] else x)
dataset['dislike']=dataset['dislike'].apply(lambda x: [new_data["movie_id"].max()+1] if x == [] else x)
train_data=dataset[(dataset.user_id >= 1)&
                                  (dataset.user_id <= 5)]
test_data=dataset[(dataset.user_id >= 6)&
                                  (dataset.user_id <= 9)]

   user_id movie_type movie_id
0        1    dislike      [3]
1        1       like      [5]
2        2       like   [2, 7]
3        3       like   [3, 8]
4        4    dislike      [8]
5        5    dislike      [4]
6        7       like   [2, 8]
7        8       like   [1, 7]
8        9    dislike      [1]
9       10    dislike      [1]


In [31]:
new_data

,movie_id,user_id,rating,unix_timestamp,all_genres,title_d,movie_type
0,1,10,2,0.291194,"3,4,5",9,dislike
1,1,9,1,0.660605,"3,4,5",9,dislike
2,1,8,4,0.519315,"3,4,5",9,like
3,2,2,3,1.000000,"1,2,16",4,like
4,2,7,5,0.000000,"1,2,16",4,like
5,3,1,1,0.181480,16,1,dislike
6,3,3,5,0.238513,16,1,like
7,4,5,2,0.212441,"1,5,8",7,dislike
8,5,1,3,0.482327,"6,8,16",3,like
9,7,2,5,0.994305,"8,15",5,like


In [32]:
train_data

,user_id,dislike,like,title_d,all_genres,unix_timestamp,predict_labels
0,1,[3],[5],"[1, 3]","[6, 8, 1]","[0.18148025805016038, 0.4823272155954916]",2
1,2,[9],"[2, 7]","[4, 5]","[6, 5, 2, 8, 1]","[1.0, 0.9943047712588169]",2
2,3,[9],"[3, 8]","[1, 6]","[4, 6, 5, 8, 1]","[0.238512862381456, 0.6559625711672339]",1
3,4,[8],[9],[6],"[4, 8, 5]",[0.22311734014859716],1
4,5,[4],[9],[7],"[8, 5, 1]",[0.2124407399567807],6


In [33]:
test_data

,user_id,dislike,like,title_d,all_genres,unix_timestamp,predict_labels
5,7,[9],"[2, 8]","[4, 6]","[4, 6, 5, 2, 8, 1]","[0.0, 0.21964208637201138]",5
6,8,[9],"[1, 7]","[9, 5]","[4, 5, 3, 8, 1]","[0.5193146561727731, 0.7694933118780622]",3
7,9,[1],[9],[9],"[4, 5, 3]",[0.6606054505013651],7


### Hyperparameter Definition

In [34]:
EMBEDDING_DIMS = 16
DENSE_UNITS = 64
DROPOUT_PCT = 0.0
ALPHA = 0.0
NUM_CLASSES=new_data["movie_id"].max() + 3
LEARNING_RATE = 0.003

### model

In [35]:
#---inputs
import tensorflow as tf
import datetime
import os
input_title = tf.keras.Input(shape=(None, ), name='movie_name')
inp_video_liked = tf.keras.layers.Input(shape=(None,), name='like')
inp_video_disliked = tf.keras.layers.Input(shape=(None,), name='dislike')
input_genre = tf.keras.Input(shape=(None, ), name='genre')
input_timestamp = tf.keras.Input(shape=(None, ), name='timestamp')


#--- layers
features_embedding_layer = tf.keras.layers.Embedding(input_dim=NUM_CLASSES, output_dim=EMBEDDING_DIMS, 
                                            mask_zero=True, trainable=True, name='features_embeddings')
labels_embedding_layer = tf.keras.layers.Embedding(input_dim=NUM_CLASSES, output_dim=EMBEDDING_DIMS, 
                                            mask_zero=True, trainable=True, name='labels_embeddings')

avg_embeddings = MaskedEmbeddingsAggregatorLayer(agg_mode='mean', name='aggregate_embeddings')

dense_1 = tf.keras.layers.Dense(units=DENSE_UNITS, name='dense_1')
dense_2 = tf.keras.layers.Dense(units=DENSE_UNITS, name='dense_2')
dense_3 = tf.keras.layers.Dense(units=DENSE_UNITS, name='dense_3')
l2_norm_1 = L2NormLayer(name='l2_norm_1')

dense_output = tf.keras.layers.Dense(NUM_CLASSES, activation=tf.nn.softmax, name='dense_output')

#--- features
features_embeddings = features_embedding_layer(input_title)
l2_norm_features = l2_norm_1(features_embeddings)
avg_features = avg_embeddings(l2_norm_features)

labels_liked_embeddings = labels_embedding_layer(inp_video_liked)
l2_norm_liked = l2_norm_1(labels_liked_embeddings)
avg_liked = avg_embeddings(l2_norm_liked)

labels_disliked_embeddings = labels_embedding_layer(inp_video_disliked)
l2_norm_disliked = l2_norm_1(labels_disliked_embeddings)
avg_disliked = avg_embeddings(l2_norm_disliked)

labels_genre_embeddings = labels_embedding_layer(input_genre)
l2_norm_genre = l2_norm_1(labels_genre_embeddings)
avg_genre = avg_embeddings(l2_norm_genre)

labels_timestamp_embeddings = labels_embedding_layer(input_timestamp)
l2_norm_timestamp = l2_norm_1(labels_timestamp_embeddings)
avg_timestamp = avg_embeddings(l2_norm_timestamp)


# 임베딩 벡터들 연결
concat_inputs = tf.keras.layers.Concatenate(axis=1)([avg_features,
                                                     avg_liked,
                                                     avg_disliked,
                                                     avg_genre,
                                                     avg_timestamp
                                                     ])
# Dense Layers
dense_1_features = dense_1(concat_inputs)
dense_1_relu = tf.keras.layers.ReLU(name='dense_1_relu')(dense_1_features)
dense_1_batch_norm = tf.keras.layers.BatchNormalization(name='dense_1_batch_norm')(dense_1_relu)

dense_2_features = dense_2(dense_1_relu)
dense_2_relu = tf.keras.layers.ReLU(name='dense_2_relu')(dense_2_features)
# dense_2_batch_norm = tf.keras.layers.BatchNormalization(name='dense_2_batch_norm')(dense_2_relu)

dense_3_features = dense_3(dense_2_relu)
dense_3_relu = tf.keras.layers.ReLU(name='dense_3_relu')(dense_3_features)
dense_3_batch_norm = tf.keras.layers.BatchNormalization(name='dense_3_batch_norm')(dense_3_relu)
outputs = dense_output(dense_3_batch_norm)

#Optimizer
optimiser = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

#--- prep model
model = tf.keras.models.Model(
    inputs=[input_title, 
            inp_video_liked, 
            inp_video_disliked,
            input_genre,
            input_timestamp,
            ],
    outputs=[outputs]
)
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
model.compile(optimizer=optimiser, loss='sparse_categorical_crossentropy', metrics=['acc'])

# model.summary()

### 학습

In [36]:
history = model.fit([tf.keras.preprocessing.sequence.pad_sequences(train_data['title_d'])+ 1e-10,
           tf.keras.preprocessing.sequence.pad_sequences(train_data['like'])+ 1e-10,
           tf.keras.preprocessing.sequence.pad_sequences(train_data['dislike'])+ 1e-10,
            tf.keras.preprocessing.sequence.pad_sequences(train_data['all_genres'])+ 1e-10,
            tf.keras.preprocessing.sequence.pad_sequences(train_data['unix_timestamp'], dtype=float) + 1e-10,
           ],train_data['predict_labels'].values,
           steps_per_epoch=1, epochs=100)

Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 2.8575 - acc: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 3ms/step - loss: 1.3783 - acc: 0.6000
Epoch 3/100
1/1 [==============================] - 0s 4ms/step - loss: 0.8500 - acc: 1.0000
Epoch 4/100
1/1 [==============================] - 0s 3ms/step - loss: 0.5791 - acc: 1.0000
Epoch 5/100
1/1 [==============================] - 0s 3ms/step - loss: 0.4222 - acc: 1.0000
Epoch 6/100
1/1 [==============================] - 0s 4ms/step - loss: 0.3240 - acc: 1.0000
Epoch 7/100
1/1 [==============================] - 0s 4ms/step - loss: 0.2572 - acc: 1.0000
Epoch 8/100
1/1 [==============================] - 0s 4ms/step - loss: 0.2096 - acc: 1.0000
Epoch 9/100
1/1 [==============================] - 0s 3ms/step - loss: 0.1746 - acc: 1.0000
Epoch 10/100
1/1 [==============================] - 0s 4ms/step - loss: 0.1466 - acc: 1.0000
Epoch 11/100
1/1 [==============================] - 0s 3ms/step - loss: 0.12

1/1 [==============================] - 0s 3ms/step - loss: 0.0029 - acc: 1.0000
Epoch 90/100
1/1 [==============================] - 0s 2ms/step - loss: 0.0029 - acc: 1.0000
Epoch 91/100
1/1 [==============================] - 0s 4ms/step - loss: 0.0029 - acc: 1.0000
Epoch 92/100
1/1 [==============================] - 0s 4ms/step - loss: 0.0028 - acc: 1.0000
Epoch 93/100
1/1 [==============================] - 0s 4ms/step - loss: 0.0028 - acc: 1.0000
Epoch 94/100
1/1 [==============================] - 0s 4ms/step - loss: 0.0027 - acc: 1.0000
Epoch 95/100
1/1 [==============================] - 0s 4ms/step - loss: 0.0027 - acc: 1.0000
Epoch 96/100
1/1 [==============================] - 0s 2ms/step - loss: 0.0026 - acc: 1.0000
Epoch 97/100
1/1 [==============================] - 0s 3ms/step - loss: 0.0026 - acc: 1.0000
Epoch 98/100
1/1 [==============================] - 0s 3ms/step - loss: 0.0026 - acc: 1.0000
Epoch 99/100
1/1 [==============================] - 0s 3ms/step - loss: 0.0025 - ac

In [37]:
results = model.evaluate([tf.keras.preprocessing.sequence.pad_sequences(test_data['title_d'])+ 1e-10,
           tf.keras.preprocessing.sequence.pad_sequences(test_data['like'])+ 1e-10,
           tf.keras.preprocessing.sequence.pad_sequences(test_data['dislike'])+ 1e-10,
            tf.keras.preprocessing.sequence.pad_sequences(test_data['all_genres'])+ 1e-10,
            tf.keras.preprocessing.sequence.pad_sequences(test_data['unix_timestamp'], dtype=float) + 1e-10,
           ], test_data['predict_labels'].values+ 1e-10, verbose=1
        )

1/1 [==============================] - 1s 810ms/step - loss: 2.8154 - acc: 0.0000e+00


In [38]:
pred = model.predict([tf.keras.preprocessing.sequence.pad_sequences(test_data['title_d'])+ 1e-10,
           tf.keras.preprocessing.sequence.pad_sequences(test_data['like'])+ 1e-10,
           tf.keras.preprocessing.sequence.pad_sequences(test_data['dislike'])+ 1e-10,
            tf.keras.preprocessing.sequence.pad_sequences(test_data['all_genres'])+ 1e-10,
            tf.keras.preprocessing.sequence.pad_sequences(test_data['unix_timestamp'], dtype=float) + 1e-10
           ])

In [39]:
pred

array([[0.073578  , 0.38703418, 0.10168213, 0.05729582, 0.0763125 ,
        0.06615589, 0.03243411, 0.0467763 , 0.04826711, 0.05056896,
        0.05989508],
       [0.07978638, 0.21397863, 0.1676723 , 0.06435131, 0.09424306,
        0.07516732, 0.05944099, 0.05321389, 0.0671754 , 0.05599924,
        0.06897144],
       [0.08010896, 0.22326857, 0.18015091, 0.06842457, 0.08035624,
        0.08048677, 0.05664931, 0.05044254, 0.06212289, 0.05826762,
        0.05972162]], dtype=float32)

In [40]:
# ranking
###### 각 user당 top-3개의 추천 데이터를 뽑아낸다.
N = 3
k = np.sort((-pred).argsort()[:,:N])
k[k>new_data["movie_id"].max()]=0
print(k)

[[1 2 4]
 [1 2 4]
 [1 2 5]]
